In [ ]:
SYFT_VERSION = ">=0.8.1b0,<=0.9"
package_string = f'"syft{SYFT_VERSION}"'
%pip install {package_string} -f https://whls.blob.core.windows.net/unstable/index.html -q

In [ ]:
import syft as sy
sy.requires(SYFT_VERSION)
from syft.client.api import NodeView
from syft.service.project.project import ProjectRequest

In [ ]:
node = sy.orchestra.launch(name="test-domain-1", port="auto", dev_mode=True)

In [ ]:
guest_domain_client = node.client

In [ ]:
guest_credentials = guest_domain_client.credentials
guest_credentials

In [ ]:
guest_domain_client.login(email="jane@caltech.edu", password="abc123")

In [ ]:
assert guest_domain_client.credentials != guest_credentials

In [ ]:
results = guest_domain_client.api.services.dataset.get_all()
results

In [ ]:
assert len(results) == 1

In [ ]:
dataset = results[-1]

In [ ]:
dataset

In [ ]:
asset = dataset.assets[0]
asset

In [ ]:
mock = asset.mock
mock

In [ ]:
mock["Trade Value (US$)"].sum()

In [ ]:
@sy.syft_function(input_policy=sy.ExactMatch(trade_data=asset.pointer),
                  output_policy=sy.SingleExecutionExactOutput())
def sum_trade_value_mil(trade_data):
    import pandas as pd
    from opendp.mod import enable_features
    enable_features('contrib')
    from opendp.measurements import make_base_laplace
    aggregate = 0.
    base_lap = make_base_laplace(scale=5.)
    noise = base_lap(aggregate)

    df = trade_data
    total = df["Trade Value (US$)"].sum()
    return (float(total / 1_000_000), float(noise))

In [ ]:
result = sum_trade_value_mil(trade_data=mock)
result

In [ ]:
assert result[0] == 9.738381

In [ ]:
assert isinstance(result[1], float)

In [ ]:
assert len(sum_trade_value_mil.kwargs) == 1

In [ ]:
node_view = NodeView.from_api(guest_domain_client.api)
assert node_view in sum_trade_value_mil.kwargs

In [ ]:
assert "trade_data" in sum_trade_value_mil.kwargs[node_view]

In [ ]:
assert sum_trade_value_mil.input_policy_init_kwargs[node_view]["trade_data"] == asset.pointer.id

In [ ]:
sum_trade_value_mil.code

In [ ]:
new_project = sy.Project(
    name="My Cool UN Project",
    description="""Hi, I want to calculate the trade volume in million's with my cool code.""",
    shareholders=[guest_domain_client],
    members=[guest_domain_client]
)
new_project

In [ ]:
new_project.create_code_request(obj=sum_trade_value_mil, client=guest_domain_client)

In [ ]:
project = new_project.start()
project

In [ ]:
project.events

In [ ]:
project.requests

In [ ]:
assert len(project.events) == 1
assert len(project.requests) == 1

In [ ]:
assert isinstance(project.events[0], ProjectRequest)

In [ ]:
result = guest_domain_client.code.sum_trade_value_mil(trade_data=asset)
result

In [ ]:
assert isinstance(result, sy.SyftNotReady)

In [ ]:
if node.node_type.value == "python":
    node.land()